In [5]:
import pandas as pd
import numpy as np
import yfinance as yf

def calculate_rsi(series, period=14):
    delta = series.diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(window=period, min_periods=1).mean()
    avg_loss = loss.rolling(window=period, min_periods=1).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def calculate_stochastic(high, low, close, period=14):
    lowest_low = low.rolling(window=period, min_periods=1).min()
    highest_high = high.rolling(window=period, min_periods=1).max()
    stochastic = 100 * ((close - lowest_low) / (highest_high - lowest_low))
    return stochastic

def calculate_williams_r(high, low, close, period=14):
    highest_high = high.rolling(window=period, min_periods=1).max()
    lowest_low = low.rolling(window=period, min_periods=1).min()
    williams_r = -100 * ((highest_high - close) / (highest_high - lowest_low))
    return williams_r

ticker = "NVDA"
stock_data = yf.download(ticker, period="2y")

stock_data['RSI'] = calculate_rsi(stock_data['Close'])
stock_data['Stochastic'] = calculate_stochastic(stock_data['High'], stock_data['Low'], stock_data['Close'])
stock_data['Williams %R'] = calculate_williams_r(stock_data['High'], stock_data['Low'], stock_data['Close'])

def calculate_trading_signals(stock_data):
    signals = []
    for i in range(len(stock_data)):
        signal = 0
        if stock_data['RSI'][i] < 30 and stock_data['Stochastic'][i] < 20 and stock_data['Williams %R'][i] < -80:
            signal = 1
        if stock_data['RSI'][i] > 70 and stock_data['Stochastic'][i] > 80 and stock_data['Williams %R'][i] > -20:
            signal = -1
        signals.append(signal)
    stock_data['Signal'] = signals
    stock_data['Prediction'] = stock_data['Signal'].diff()
    return stock_data

stock_data = calculate_trading_signals(stock_data)

def calculate_returns(stock_data, initial_capital=100000):
    capital = initial_capital
    capital_series = [capital]
    holding = False
    buy_price = 0

    for i in range(len(stock_data)):
        if stock_data['Prediction'][i] == 1:
            buy_price = stock_data['Close'][i]
            holding = True
        elif stock_data['Prediction'][i] == -1 and holding:
            capital += (stock_data['Close'][i] - buy_price) * (capital / buy_price)
            holding = False
        capital_series.append(capital)

    stock_data['Capital'] = capital_series[1:]
    return stock_data

stock_data = calculate_returns(stock_data)
final_capital = stock_data['Capital'].iloc[-1]

def calculate_max_drawdown(stock_data):
    cumulative = stock_data['Capital']
    running_max = cumulative.cummax()
    drawdown = cumulative - running_max
    max_drawdown = drawdown.min()
    return max_drawdown*(-1)

max_drawdown = calculate_max_drawdown(stock_data)

def calculate_sharpe_ratio(stock_data, risk_free_rate=0):
    daily_returns = stock_data['Capital'].pct_change().dropna()
    excess_return = daily_returns - risk_free_rate/252
    mean_excess_return = excess_return.mean()
    std_excess_return = excess_return.std()
    sharpe_ratio = mean_excess_return / std_excess_return
    return sharpe_ratio * np.sqrt(252)

sharpe_ratio = calculate_sharpe_ratio(stock_data)

print(f"Final Capital:₹{final_capital:.2f}")
print(f"Maximum Drawdown:₹{max_drawdown:.2f}")
print(f"Sharpe Ratio:{sharpe_ratio:.2f}")


[*********************100%%**********************]  1 of 1 completed

Final Capital:₹746135.99
Maximum Drawdown:₹12010.69
Sharpe Ratio:3.05
